一、选择预测目标

1. 择时作为主要目标

    交易无非选股与择时，选股是一种更高层次的择时，因子的有效性也不是永久的，无论是选择资产还是选择因子，都是期望选择出在未来一段时间更为强势的资产、因子。

    由于重点研究的是股指期货，所以基本上也不存在选股的问题，最多只是在IF 和 IC中间做一下选择。
    
    因此整个体系重点研究择时，通过预期资产未来一段时间的变化，给出交易建议

2. 预测价格变动，而不是预测价格本身
    
    很多学术的时间序列预测都是对时间序列本身进行建模，这样的方式在对价格进行建模上会出现严重问题：
    
    看上去预测线跟实际线走势非常好看，但是一回测收益就一塌糊涂

    价格是具有强烈单位根特点的时间序列数据，直接进行建模需要严格的时间序列处理，而深度学习处理这些问题是比较棘手的。
    
    直接利用深度学习模式对价格建模很容易出现模型表现很好，损失很小，类似于伪回归，而忽略的方向的重要性，回测一塌糊涂的现象。

    因此，必须把预测目标转向一阶差分，即价格的变动上，价格的变动不仅是平稳的，有良好的时间序列性质，而且更具有实际意义，可以直接用于交易

3. 预测未来5-10天的价格
    
    那么应该预测多久的未来变动呢？
    
    预测的时间越短，噪音占比越高，随机扰动、市场情绪都会对第价格产生巨大影响，很难区分真正的信号的作用；

    预测的时间越长，宏观因素和黑天鹅事件的影响力越大，价格的变动更可能取决与强大的无法预测的外部宏观因素；

    因此，无论太长或者太短都是不合适的。模型需要保持策略与信息的时间一致性。
    
    一个高频交易的策略，应该把分钟线、盘口情况、实时新闻加入模型的特征，而不应该关注未来半年的宏观预期；

    反过来一个长线投资的交易者也不可能关注短期的日内扰动。因此需要某种程度上的“知行合一”

    5-10日的预测长度，主要有以下三个优点：

    (1). 大幅度减少了日内扰动的影响

    (2). 5-10天基本面的变化不会特别的大，即使真的遇到了特别大的基本面变动，也有足够的时间去人为干预

    (3). 有一些时序模型，如Informer等有较强的多步预测能力，结合的日频数据进行5-10步的多步预测可以充分发挥模型潜力

In [6]:
import pandas as pd
data = pd.read_csv('data/market_state.csv', index_col='Unnamed: 0')
print(f"未来1天价格的每日收益标准差：{data['label_ic_ch_next_day'].describe()['std']:.2f}")
print(f"未来5天价格的每日收益标准差：{data['label_ic_ch_next_week'].describe()['std']/5:.2f}")
print(f"未来10天价格的每日收益标准差：{data['label_ic_ch_next_hfmonth'].describe()['std']/10:.2f}")

未来1天价格的每日收益标准差：80.35
未来5天价格的每日收益标准差：36.78
未来10天价格的每日收益标准差：24.99


可以看到，通过5日移动平均的方式，大幅减少了预测目标的标准差，等效降低了噪声的影响，更有利于提高夏普比率。

但预测窗口的长度并非可以无限提升，预测窗口越长，预测窗口到预测精度也会越低；

同时，降低标准差的作用也会越来越微弱，整体平衡下来，5-10日的预测窗口是比较合适的选择。

此外，还有一个关于多日预测的小问题：如果第一天预测未来5天是上涨的开了多仓，如果第二天方向变了怎么办？可以通过滚动持仓的形式来实现预测结果的实现，例如:

Day 1 20%仓位，执行第一天的投资决策

Day 2 20%仓位，执行第二天的投资决策

...

Day 6 第一天的20%仓位到期，执行第六天的投资决策

通过这种方式，可以实现用每日的交易兑现未来5天预期，保证预测结果是可交易的，且避免出现信号方向频繁变动带来的问题。

二、特征

由于金融数据的噪声占比很大，直接拟合原始数据将收到噪声的严重干扰。

将大部分可以收集到的信息都加入信息矩阵的维度固然可以快速在训练集上降低损失，但是这会带来严重的过拟合问题

一个有效的指标，应该可以同时降低训练集和测试集的损失；而无效的指标只会让模型更容易记住训练集的数据

我们可以借助XGBoost快速筛选维度：

在现有的维度上，进行一次XGBoost拟合，记录训练集和测试集损失，测试集损失会略高一些，这是必然的；

添加新的维度，用同样的参数进行XGBoost拟合，观察训练集和测试集损失，显然训练集损失一定会下降，但测试集损失并不一定；

如果测试集损失也下降，至少说明新加入的维度带来的信息是有效的，而不是无效信息。

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import tqdm
from sklearn.metrics import mean_squared_error

def evaluate_new_feature(data, base_features, new_features, target, n_estimators = 5, m_round = 5, test_ratio = 0.2):
    """
    通过多次随机的数据划分，评估新特征的效果
    """
    base_test_losses = []
    new_test_losses = []

    for i in range(m_round):
        X = data[base_features + new_features]
        y = data[target]

        test_size = int(len(X)*test_ratio)
        random_start = np.random.randint(test_size, len(X) -  test_size)
        X_train, X_test = X[:random_start], X[random_start:]
        y_train, y_test = y[:random_start], y[random_start:]

        base_model = xgb.XGBRegressor(n_estimators=n_estimators)
        base_model.fit(X_train[base_features], y_train)
        base_test_preds = base_model.predict(X_test[base_features])
        base_test_loss = mean_squared_error(y_test, base_test_preds)
        base_test_losses.append(base_test_loss)

        new_model = xgb.XGBRegressor(n_estimators=n_estimators)
        new_model.fit(X_train, y_train)
        new_test_preds = new_model.predict(X_test)
        new_test_loss = mean_squared_error(y_test, new_test_preds)
        new_test_losses.append(new_test_loss)

    avg_base_test_loss = np.mean(base_test_losses)
    avg_new_test_loss = np.mean(new_test_losses)

    return avg_base_test_loss, avg_new_test_loss

best n_estimators 相当于是评估信息在这个因子中被埋藏的深度：

如果一个因子在n_estimators = 5的时候就达到最佳损失下降的目的，便说明这个因子在很小的模型上就能发挥作用；

但如果一个因子的best n_estimators = 100，说明信息在这个因子中埋藏的很深，需要很大的神经网络才能完全利用相关信息，可以尝试进一步优化；

而如果平均下降幅度不大，则说明这个信息可能没有用

In [ ]:
def find_best_n_estimators(data, base_features, new_features, target, n_estimators_range, m_round = 5, test_ratio=0.2):
    """
    通过测试不同的n_estimators，寻找能带来最大测试损失下降幅度的新特征模型。
    returns:  best_n_estimators, max_reduction, mean_reduction
    """
    data_copy = data.dropna()
    results = {}
    max_reduction = 0
    best_n_estimators = -1
    reductions = []

    for n_est in tqdm.tqdm(n_estimators_range):
        avg_base_loss, avg_new_loss = evaluate_new_feature(
            data=data_copy,
            base_features=base_features,
            new_features=new_features,
            target=target,
            n_estimators=n_est,
            m_round=m_round,
            test_ratio=test_ratio,
        )
        
        # 计算损失下降幅度
        loss_reduction_rate = (avg_base_loss - avg_new_loss) / avg_base_loss
        results[n_est] = loss_reduction_rate
        reductions.append(loss_reduction_rate)
        if loss_reduction_rate > max_reduction:
            max_reduction = loss_reduction_rate
            best_n_estimators = n_est

    mean_reduction = np.mean(reductions)
    return best_n_estimators, max_reduction, mean_reduction

In [ ]:
data = pd.read_csv('data/market_state.csv', index_col='Unnamed: 0')
data['open_ch'] = (data['open_ih'] - data['close_ih'].shift(1)) /  data['close_ih'].shift(1)
data['high_ch'] = (data['high_ih'] - data['close_ih'].shift(1)) /  data['close_ih'].shift(1)
data['low_ch'] = (data['low_ih'] - data['close_ih'].shift(1)) /  data['close_ih'].shift(1)
data['close_ch'] = (data['close_ih'] - data['close_ih'].shift(1)) /  data['close_ih'].shift(1)
data

In [ ]:
base_features = ['open_ch', 'high_ch', 'low_ch', 'close_ch','near_maturity','on', 'm1','y1', 'y10','near_discount_ih','far_discount_ih','%_diff_ih_5','%_diff_ih_20','%_diff_ih_60','MACD_ih',]
new_features = ['RS_ih']

for i in base_features + new_features:
    data[i] =  (data[i] - data[i].mean())/data[i].std()

target = 'label_ih_ch_next_week'

n_estimators_range = range(1,50,2)
m_round = 10

# 测试变量
best_n, max_reduction, mean_reduction = find_best_n_estimators(
    data = data,
    base_features = base_features,
    new_features = new_features,
    target = target,
    n_estimators_range = n_estimators_range,
    m_round = m_round,
    test_ratio=0.3
)

print(f"\n最佳的拟合次数为 {best_n}，在此次数下，新特征可以降低测试损失 {max_reduction:.2%}, 平均可以降低 {mean_reduction:.2%}")

当我们确定了一个因子有效之后，还可以通过xgboost优化这个因子的参数，例如MACD中平均线的窗口长度等等。

In [ ]:
from itertools import product
def grid_search_feature_optimization(data, base_features, target, feature_gen_func, param_ranges, n_estimators_range, m=5, test_size=0.2, random_state=42):
    """
    通过网格搜索，寻找能带来最大损失下降幅度的特征生成参数组合。

    参数:
    - data (DataFrame): 原始数据集。
    - base_features (list): 原始特征的列名。
    - target (str): 目标变量的列名。
    - feature_gen_func (function): 用户自定义的特征生成函数。
    - param_ranges (dict): 参数名和其搜索范围的字典，如 {'param1': [1, 2], 'param2': [3, 4]}。
    - n_estimators_range (list): 待测试的n_estimators值的列表。
    - m (int): 每次评估的数据划分次数。
    - test_size (float): 测试集所占的比例。
    - random_state (int): 用于控制随机性的种子。

    返回:
    - tuple: (最佳参数组合, 最佳n_estimators, 最大损失下降幅度)
    """

    param_names = list(param_ranges.keys())
    param_values = list(param_ranges.values())
    
    best_params = None
    best_n_estimators_for_params = None
    max_overall_reduction = -float('inf')

    print("=====================================================")
    print("--- 正在进行特征参数的网格搜索 ---")
    print(f"搜索参数: {param_ranges}")
    print("=====================================================\n")

    # 遍历所有参数组合
    for params in product(*param_values):
        current_params_dict = dict(zip(param_names, params))
        print(f"当前评估参数组合: {current_params_dict}")
        
        # 使用当前参数生成新特征
        temp_data = data.copy()
        
        # **重要**: 在这里，我们调用用户定义的特征生成函数
        new_data = feature_gen_func(temp_data, **current_params_dict)
        
        # 识别新生成的特征
        new_features = [col for col in new_data.columns if col not in data.columns]
        
        if not new_features:
            print("警告: 当前参数组合没有生成新特征，跳过。")
            continue

        # 调用 find_best_n_estimators 来评估这个新特征组合
        best_n, max_reduction_for_params = find_best_n_estimators(
            data=new_data,
            base_features=base_features,
            new_features=new_features,
            target=target,
            n_estimators_range=n_estimators_range,
            m=m,
            test_size=test_size,
            random_state=random_state
        )

        # 检查是否为迄今为止的最佳表现
        if max_reduction_for_params > max_overall_reduction:
            max_overall_reduction = max_reduction_for_params
            best_params = current_params_dict
            best_n_estimators_for_params = best_n
        
        print("\n" + "=" * 80 + "\n") # 大分隔线

    print("=====================================================")
    print("--- 最终最佳参数组合 ---")
    print(f"最佳特征生成参数: {best_params}")
    print(f"对应的最佳拟合次数: {best_n_estimators_for_params}")
    print(f"带来的最大平均测试损失下降幅度: {max_overall_reduction:.4f}")
    print("=====================================================")
    
    return best_params, best_n_estimators_for_params, max_overall_reduction

In [ ]:
# # 1. 创建一个示例数据集
# data = {'feature_A': np.random.rand(200),
#         'feature_B': np.random.rand(200),
#         'feature_C': np.random.rand(200),
#         'target': np.random.rand(200)}
# df = pd.DataFrame(data)

# # 2. 定义你的特征生成函数
# def feature_generator_func(data, param1, param2):
#     data['new_feature_custom'] = data['feature_A'] * param1 + data['feature_B'] / param2
#     return data

# # 3. 定义参数的搜索范围
# param_ranges = {
#     'param1': [1, 2, 3],
#     'param2': [10, 20]
# }

# # 4. 定义需要测试的拟合次数范围
# n_est_candidates = [50, 100, 200]

# # 5. 定义原始特征和目标变量
# base_feats = ['feature_A', 'feature_B', 'feature_C']
# target_col = 'target'

# # 6. 调用网格搜索函数
# best_params, best_n, max_reduction = grid_search_feature_optimization(
#     data=df,
#     base_features=base_feats,
#     target=target_col,
#     feature_gen_func=feature_generator_func,
#     param_ranges=param_ranges,
#     n_estimators_range=n_est_candidates
# )

# print("\n")
# print("-" * 50)
# print("网格搜索最终结果:")
# print(f"最佳参数组合: {best_params}")
# print(f"最佳拟合次数: {best_n}")
# print(f"最大损失下降幅度: {max_reduction:.4f}")
# print("-" * 50)